In [1]:
import numpy as np
import sys, os
funcPath = '/Users/srothmei/Documents/TVB-Pypeline'
sys.path.append(funcPath)

import bm_functions as brainmodes

debugPath = '/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/'

In [2]:
wmBorder_file = debugPath + 'wmborder.npy'

#wmborder = np.load(wmBorder_file)
wmborder = wmBorder_file

In [3]:
affine_matrix_file = debugPath + 'affine_matrix.npy'

affine_matrix = np.load(affine_matrix_file)

In [4]:
roi = 1
subid = 'Adalberto'
tracksPath = debugPath

In [5]:
from nipype import Node
from nipype.interfaces.io import DataFinder
tckFinder = Node(DataFinder(match_regex = '.*\.npy', root_paths = tracksPath), name = 'tckFinder')

res = tckFinder.run()
track_files = res.outputs.out_paths

INFO:workflow:Executing node tckFinder in dir: /var/folders/tg/2g576vk174s3m9znw1sgjpbm0000gn/T/tmpuC7E7W/tckFinder


In [6]:
brainmodes.compute_connectivity_row(roi, subid, affine_matrix, wmborder, tracksPath, track_files)

Computing SC for ROI 1
Generate ROI-ID to voxel hashtable...
/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/10011_tracks.npy: Tracks loaded .....
/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/10012_tracks.npy: Tracks loaded .....
/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/10013_tracks.npy: Tracks loaded .....
Storing....
Done!


('/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/SC_cap_row_1Adalberto.json',
 '/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/SC_dist_row_1Adalberto.json')

In [ ]:
    region_table = np.unique(wmborder[wmborder > 0]).astype(int)
    region_table = region_table.tolist()
    print('Test')
    # Generate ROI-ID to voxel hashtable
    print('Generate ROI-ID to voxel hashtable...')
    region_id_table = np.array((0, 0)).astype('int16')  # Init Variable
    for regid in region_table:
        tmpids = np.ravel_multi_index(np.nonzero(wmborder == regid), wmborder.shape, order='F')
        tmpids.sort()
        tmpids = np.vstack((np.ones_like(tmpids) * regid, tmpids))
        region_id_table = np.vstack((region_id_table, np.transpose(tmpids)))
    region_id_table = region_id_table[1:, :].astype(int)

In [ ]:
import re
from os.path import basename
tileFiles = list()
for trackFile in track_files:
    #print(basename(trackFile))
        # Select only files that have max. of 2 trailing number depicting the ordering...
        # i.e. don't select files like '10012_subID.tck' when processing for region 10...
    if re.search("^" + str(region_table[roi - 1]) + "\d{1,2}_.*\.npy$", basename(trackFile)):
        tileFiles.append(trackFile)
            
print type(tileFiles)
print tileFiles[0]

In [ ]:
region_id_table[1:10,:]

In [ ]:
def tck2voxel_cluster(tck, affineMatrix):
    from numpy import hstack, round, dot, ones, shape, transpose

    # Loop over all tracks in the data structure
    for i in range(shape(tck)[0]):
        # Transform the coordinates by multiplying the affine matrix with the coords
        # => First add a column of 1 to the right side of the matrix
        zw = hstack((tck[i], ones((shape(tck[i])[0], 1))))
        # => Second multiply the actual matrices
        zw = round(dot(zw, transpose(affineMatrix)))
        # => Third store the result excluding the righthand-side column
        tck[i] = zw[:, :3].astype('int16')

    return tck

tile = tileFiles[0]
tracks_header = np.load(tile).item()
tracks_header['tracks'] = tck2voxel_cluster(tracks_header['tracks'], affine_matrix)


In [ ]:
for trackind in range(np.shape(tracks_header['tracks'])[0]):
    inRegIDs = wmborder[tracks_header['tracks'][trackind][:, 0], tracks_header['tracks'][trackind][:, 1], tracks_header['tracks'][trackind][:, 2]]
    print inRegIDs
    inRegLin = np.flatnonzero(inRegIDs[:-1])
    print np.flatnonzero(inRegIDs[:-1])
    pathIndices = np.ravel_multi_index(
                    (tracks_header['tracks'][trackind][:, 0], tracks_header['tracks'][trackind][:, 1], tracks_header['tracks'][trackind][:, 2]), np.shape(wmborder),
                    order='F')
    print pathIndices
    break

In [ ]:
foo = [np.where(region_id_table[:,1] == x)[0][0] for x in pathIndices if np.where(region_id_table[:,1] == x)[0]]
inRegIDs = region_id_table[foo,0]
print inRegIDs
print foo
inRegLin2 = range(len(region_id_table[foo,0])-1)
print np.flatnonzero(region_id_table[:, 1] == pathIndices[inRegLin2[-1]])[0]
print foo[inRegLin2[-1]]

In [ ]:
if np.where(region_id_table[:,1] == pathIndices[0])[0]:
    print "Besart"

In [ ]:
wmborder = "/Users/srothmei/Desktop/charite/temp/wmborder.npy"
print type(wmborder)
wmborder = np.load(wmborder)
print type(wmborder)
region_table = np.unique(wmborder[wmborder > 0]).astype('int32')
region_table = region_table.tolist()

print type(int(np.count_nonzero(wmborder == region_table[roi - 1]) * 200))
qualityMetrics = {'off_seed': 0, 'too_short': 0, 'good_tracks': 0, 'wrong_seed': 0,
                      'expected_tracks': np.count_nonzero(wmborder == region_table[roi - 1]) * 200, 'wrong_target': 0,
                      'generated_tracks': 0}
print qualityMetrics

In [ ]:
region_table[roi -1]